# Using load confounds with scrubbing

This tutorial explains how to use load_confounds with th scrubbing option

With a scrubbing-based strategy, load_confounds returns a sample_mask that removes the index of volumes exceeding the framewise displacement and standardised DVARS threshold, and all the continuous segment with less than five volumes.<br/> **Before applying scrubbing, it’s important to access the percentage of volumns scrubbed. Scrubbing is not a suitable strategy for datasets with too many high motion subjects.** <br/> 
Read more: [load_confounds](https://nilearn.github.io/stable/modules/generated/nilearn.interfaces.fmriprep.load_confounds.html?highlight=load_confounds#footcite-power2014)

Download these packeges if you don't have them in you virtual envioroment

In [1]:
# Download these packeges if you don't have them in you virtual envioroment. (Un-comment code)

import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install nibabel
# !{sys.executable} -m pip install nilearn
# !{sys.executable} -m pip install pandas

Import needed moduels 

In [2]:
import pandas as pd
import nibabel as nib
from nilearn.maskers import NiftiMasker
from nilearn import image as nimg
from nilearn import masking

For using load confounds you have to load an fmriprep output functionl file. For most purpuses, if <br/>
you are not intrested in using the aroma confounds non-aggressivley, you should use the fmriprep <br/>
output file that ends with the suffix **"desc-preproc_bold.nii.gz"**


In [3]:
#Load the path of the functional file according to you enviroment's path
func_file =f"fmriprep/derivative-sub-Y01/sub-Y01/ses-1/func/sub-Y01_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz"

#Load the fmpriprep mask img
brain_mask = f"fmriprep/derivative-sub-Y01/sub-Y01/anat/sub-Y01_space-MNI152NLin2009cAsym_desc-brain_mask.nii.gz"

load_confound returns a confounds dataframe and a sample mask.<br/>
The confounds dataframe includes the regressors (confound colomns) selected with the spesefied strategy when calling the funnction.<br/>
**The sample mask** is a mask for the non-study state volumes. This attribute should be passed to parameter sample_mask of nilearn.maskers.NiftiMasker or nilearn.signal.clean. <br/>
The sample mask is espitially important when using the scrubbing opttion as it specifies which voxels the index of volumes exceeding the framewise displacement.

#### **Load confounds parameters with scrubbing**

Make sure to include "scrub" in your strategy and specify the associated parameter: scrub, fd_threshold, std_dvars_threshold.

**scrub** int, default 5: <br/>
After accounting for time frames with excessive motion, further remove segments shorter than the given number. The default value is 5 (referred as full scrubbing in [this paper](https://www.sciencedirect.com/science/article/pii/S1053811913009117)). When the value is 0, temove time frames based on excessive framewise displacement and DVARS only. One-hot encoding vectors are added as regressors for each scrubbed frame.<br/>
**fd_threshold** float, default 0.2:  <br/>
Framewise displacement threshold for scrub (default = 0.2 mm): <br/>
**std_dvars_threshold** float, default 3:<br/>
Standardized DVARS threshold for scrub (default = 3).
DVARs is defined as root mean squared intensity difference of volume N to volume N+1 4. D referring to temporal derivative of timecourses, VARS referring to root mean squared variance over voxels.<br/>



In [4]:
#Load the module that allows for flexible paramters strategy 
from nilearn.interfaces.fmriprep import load_confounds

#Extract confounds from fmripreps tsv confounds file
confounds, sample_mask = load_confounds( func_file ,strategy=["motion", "wm_csf", "scrub"], motion="basic", wm_csf="basic", 
                                                          scrub=5, fd_threshold=0.2, std_dvars_threshold=3)

In [5]:
confounds.head()

,csf,rot_x,rot_y,rot_z,trans_x,trans_y,trans_z,white_matter
0,6.563275,-0.001496,-0.001536,-0.000417,0.030476,-0.058516,-0.098625,-0.198772
1,2.642220,-0.001702,-0.001649,-0.000417,0.035698,0.036520,-0.058892,-0.291134
2,-0.565720,-0.001526,-0.001536,-0.000417,0.035863,-0.099453,-0.091586,-0.343848
3,-2.643851,-0.001493,-0.001353,-0.000417,0.040728,-0.031964,-0.070679,-0.609428
4,-1.325184,-0.001642,-0.001353,-0.000417,0.040728,0.007792,-0.079698,0.272923


Let's take a look at tha sample mask <br/>
shape: (number of scans - number of volumes removed, ) <br>
Meaning - the indecies of the niimgs along time/fourth dimension for **valid volumes for subsequent analysis**.

In [6]:
print("Indecies of valid volumes:")
print(sample_mask)

Indecies of valid volumes:
[  0   1   2   3   4   5  19  20  21  22  23  24  25  60  61  62  63  64
  65  66  86  87  88  89  90  91  92  93  94  95 113 114 115 116 117 168
 169 170 171 172 173 174 175 187 188 189 190 191 192 193 197 198 199 200
 201 202 203]


In [7]:
sample_mask.shape 

(57,)

So, only 57 volumes are valid!<br/>
Keeping in mind that:

In [8]:
print(f"Total number of volumes is {len(confounds.index)}")

Total number of volumes is 379


In [ ]:
#Masker is used for applying a mask to extract time-series from Niimg-like objects
## Not sure wether o standardize signal
masker= NiftiMasker(mask_img=brain_mask, memory='nilearn_cache', verbose=5, t_r=1.4, low_pass=0.1, high_pass=0.01)

#Extract time series and regress out confounds using GLM
time_series = masker.fit_transform(func_file, confounds=confounds, sample_mask=sample_mask)

# Transform the 2D data matrix back to an image in brain space
clean_img = masker.inverse_transform(time_series)

[NiftiMasker.fit] Loading data from None
[NiftiMasker.fit] Resampling mask
[Memory]0.0s, 0.0min    : Loading resample_img...
________________________________________resample_img cache loaded - 0.0s, 0.0min
________________________________________________________________________________
[Memory] Calling nilearn.maskers.nifti_masker._filter_and_mask...
_filter_and_mask('fmriprep/derivative-sub-Y01/sub-Y01/ses-1/func/sub-Y01_ses-1_task-rest_run-1_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz', 
<nibabel.nifti1.Nifti1Image object at 0x7f5b1f359000>, { 'detrend': False,
  'dtype': None,
  'high_pass': 0.01,
  'high_variance_confounds': False,
  'low_pass': 0.1,
  'reports': True,
  'runs': None,
  'smoothing_fwhm': None,
  'standardize': True,
  'standardize_confounds': True,
  't_r': 1.4,
  'target_affine': None,
  'target_shape': None}, memory_level=1, memory=Memory(location=nilearn_cache/joblib), verbose=5, confounds=[            csf     rot_x     rot_y     rot_z   trans_x   trans_y

In [ ]:
nib.save(clean_img, f"tutorial/sub-01_ses-01_run-01_cleaned_scrub.nii.gz")